<a href="https://colab.research.google.com/github/Aishatoo07/AissatouDieng_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [1]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [2]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [3]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [4]:
# Q1 — IBAN Country Codes (table)
# URL: https://www.iban.com/country-codes
# Expected: data_q1.csv + Top-15 sorted by Numeric (desc)

import pandas as pd
import requests

URL = "https://www.iban.com/country-codes"
UA = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36"}

# --- Implementations for the 3 TODOs ---

def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML."""
    tables = pd.read_html(html)  # requires lxml
    if not tables:
        raise ValueError("No tables found in HTML.")

    # Flatten headers for each candidate table
    def _flatten(df: pd.DataFrame) -> pd.DataFrame:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [" ".join([str(x) for x in tup if str(x) != "nan"]).strip()
                          for tup in df.columns.values]
        else:
            df.columns = [str(c).strip() for c in df.columns]
        return df

    chosen = None
    for t in tables:
        t = _flatten(t)
        if t.shape[1] >= 3:
            chosen = t
            break
    if chosen is None:
        raise ValueError("No suitable table (>= 3 columns) found.")

    # Canonicalize header names (keep extras)
    canon = {
        "country": "Country",
        "country name": "Country",
        "alpha-2 code": "Alpha-2",
        "alpha 2 code": "Alpha-2",
        "alpha2": "Alpha-2",
        "alpha-3 code": "Alpha-3",
        "alpha 3 code": "Alpha-3",
        "alpha3": "Alpha-3",
        "numeric": "Numeric",
        "numeric code": "Numeric",
    }
    new_cols = []
    for c in chosen.columns:
        key = str(c).strip().lower()
        new_cols.append(canon.get(key, str(c).strip()))
    chosen.columns = new_cols
    return chosen


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids."""
    out = df.copy()

    # Ensure required columns exist (keep extras)
    req = ["Country", "Alpha-2", "Alpha-3", "Numeric"]
    for c in req:
        if c not in out.columns:
            out[c] = pd.NA

    # Strip whitespace on object columns
    for c in out.select_dtypes(include=["object"]).columns:
        out[c] = out[c].astype(str).str.strip()

    # Uppercase codes
    out["Alpha-2"] = out["Alpha-2"].astype(str).str.strip().str.upper().replace({"<NA>": pd.NA})
    out["Alpha-3"] = out["Alpha-3"].astype(str).str.strip().str.upper().replace({"<NA>": pd.NA})

    # Numeric -> nullable Int64
    out["Numeric"] = pd.to_numeric(out["Numeric"], errors="coerce").astype("Int64")

    # Drop invalid/blank Country
    out["Country"] = out["Country"].replace({"nan": pd.NA}).astype("string").str.strip()
    out = out.dropna(subset=["Country"]).reset_index(drop=True)

    # Put required 4 first
    ordered = [c for c in req if c in out.columns] + [c for c in out.columns if c not in req]
    return out[ordered]


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N."""
    return (
        df.sort_values(by="Numeric", ascending=False, na_position="last")
          .head(top)
          .reset_index(drop=True)
    )

# --- Fetch -> Parse -> Clean -> Save -> Print Top-15 ---
html = requests.get(URL, headers=UA, timeout=20).text
raw = q1_read_table(html)
clean = q1_clean(raw)
clean.to_csv("data_q1.csv", index=False)

top15 = q1_sort_top(clean, top=15)
print("Q1 — Top-15 by Numeric (desc):")
print(top15.to_string(index=False))
print("\nSaved: data_q1.csv")

# (Optional) Minimal README.md for your deliverables
readme = """# Assignment 6 — Q1: IBAN Country Codes
- URL: https://www.iban.com/country-codes
- Steps:
  1) Fetch page HTML with a polite User-Agent.
  2) Use `pandas.read_html` to extract the first table with ≥3 columns.
  3) Clean columns: trim whitespace; uppercase Alpha-2/Alpha-3; cast Numeric to nullable Int64.
  4) Save as `data_q1.csv`.
  5) Print Top-15 rows sorted by `Numeric` (descending).
- Limitation:
  - Relies on the site’s current table structure and header texts; if they change, header mapping may need updating.
"""
with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme)


Q1 — Top-15 by Numeric (desc):
                                                   Country Alpha-2 Alpha-3  Numeric
                                                    Zambia      ZM     ZMB      894
                                                     Yemen      YE     YEM      887
                                                     Samoa      WS     WSM      882
                                         Wallis and Futuna      WF     WLF      876
                        Venezuela (Bolivarian Republic of)      VE     VEN      862
                                                Uzbekistan      UZ     UZB      860
                                                   Uruguay      UY     URY      858
                                              Burkina Faso      BF     BFA      854
                                     Virgin Islands (U.S.)      VI     VIR      850
                            United States of America (the)      US     USA      840
                              Tanzania, Unite

/tmp/ipython-input-3967212036.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)  # requires lxml


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [13]:
import re
import pandas as pd
from bs4 import BeautifulSoup

_DIGITS = re.compile(r"(\d+)")

def _to_int(x) -> int:
    """Extract first digits from text; non-digits -> 0."""
    if x is None:
        return 0
    m = _DIGITS.search(str(x))
    return int(m.group(1)) if m else 0

def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into: rank, title, link, points, comments, user."""
    soup = BeautifulSoup(html, "lxml")
    rows = []
    for athing in soup.select("tr.athing"):
        # rank/title/link from the athing row
        rank_txt = (athing.select_one(".rank").get_text(strip=True) if athing.select_one(".rank") else "")
        title_a = athing.select_one(".titleline a")
        title = title_a.get_text(strip=True) if title_a else ""
        link = title_a.get("href", "") if title_a else ""

        # details from the immediate sibling row's .subtext
        points, comments, user = 0, 0, ""
        subrow = athing.find_next_sibling("tr")
        if subrow:
            sub = subrow.select_one(".subtext")
            if sub:
                score = sub.select_one(".score")
                points = _to_int(score.get_text(strip=True) if score else "")

                u = sub.select_one(".hnuser")
                user = u.get_text(strip=True) if u else ""

                a_tags = sub.select("a")
                if a_tags:
                    comments_txt = a_tags[-1].get_text(strip=True)  # usually "... comments" or "discuss"
                    comments = _to_int(comments_txt)

        rows.append({
            "rank": _to_int(rank_txt),
            "title": title,
            "link": link,
            "points": points,
            "comments": comments,
            "user": user
        })
    return pd.DataFrame(rows)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Cast points/comments/rank to int (non-digits->0); fill text fields."""
    out = df.copy()

    # Ensure text fields exist and are clean
    for col in ["title", "link", "user"]:
        if col not in out.columns:
            out[col] = ""
        out[col] = out[col].fillna("").astype(str).str.strip()

    # Ensure numeric fields exist and coerce to int via digit extraction
    for col in ["rank", "points", "comments"]:
        if col not in out.columns:
            out[col] = 0
        out[col] = out[col].apply(_to_int).astype(int)

    return out

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    return df.sort_values("points", ascending=False, na_position="last").head(top).reset_index(drop=True)


In [12]:
import requests

URL = "https://news.ycombinator.com/"
UA = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36"}

html = requests.get(URL, headers=UA, timeout=20).text
raw = q2_parse_items(html)
clean = q2_clean(raw)
clean.to_csv("data_q2.csv", index=False)

top15 = q2_sort_top(clean, top=15)
print("Q2 — Top-15 by points (desc):")
print(top15.to_string(index=False))
print("\nSaved: data_q2.csv")


Q2 — Top-15 by points (desc):
 rank                                                                       title                                                                                                                                                                link  points  comments         user
   13 YouTube Removes Windows 11 Bypass Tutorials, Claims 'Risk of Physical Harm'                                                                                               https://news.itsfoss.com/youtube-removes-windows-11-bypass-tutorials/     418       158  WaitWaitWha
   10                                                     Why I love OCaml (2023)                                                                                                                     https://mccd.space/posts/ocaml-the-worlds-best/     300       204        art-w
   22              VLC's Jean-Baptiste Kempf Receives the European SFS Award 2025                                                          